# Overlapとmp_term_nameの統合（離散値）

- 一例：Male infertility

In [1]:
# Move up to top directory
import os
os.getcwd()
os.chdir('../')
os.getcwd()

'/mnt/g/impc-genes_with_similar_phenotypes'

In [63]:
from pathlib import Path
from collections import defaultdict
from itertools import combinations
import csv
import pandas as pd
from matplotlib import pyplot as plt

In [64]:
path_overlap = Path("data", "overlap_ratios.csv")
path_mp = Path("data", "mp_term_name", "male_infertility.csv")

In [65]:
df_overlap = pd.read_csv(path_overlap)
df_mp = pd.read_csv(path_mp)
df_mp = df_mp[["marker_symbol", "effect_size"]].drop_duplicates()


In [66]:
print(df_overlap.head(3))
df_mp

  marker1 marker2  overlap_ratio  overlapped_mp_number  \
0    Dpf2   Gna13          0.167                     2   
1    Dpf2    Plec          0.200                     2   
2    Dpf2  Gnptab          0.080                     2   

                                       overlapped_mp  \
0  abnormal embryo size\npreweaning lethality, co...   
1   edema\npreweaning lethality, complete penetrance   
2  abnormal heart morphology\nabnormal kidney mor...   

                                          marker1_mp  \
0  abnormal craniofacial morphology\nabnormal emb...   
1  abnormal craniofacial morphology\nabnormal emb...   
2  abnormal craniofacial morphology\nabnormal emb...   

                                          marker2_mp  
0  abnormal embryo size\nabnormal visceral yolk s...  
1   edema\npreweaning lethality, complete penetrance  
2  abnormal blood vessel morphology\nabnormal hea...  


,marker_symbol,effect_size
0,Tex38,1
1,Pou2af3,1
2,Stoml2,1
3,Sbf1,1
4,Cyrib,1
...,...,...
320,Cep19,1
321,Bzw1,1
322,Septin10,1
323,Cfap57,1


In [67]:
# print(df_overlap[df_overlap["intersect_count"] > 5 & df_overlap["overlap_ratio"] > 0.5])

In [77]:
# df_overlap_filterd = df_overlap[df_overlap["overlapped_mp_number"] > 0]
df_overlap_filterd = df_overlap[(df_overlap["overlap_ratio"] > 0.5) & (df_overlap["overlapped_mp_number"] > 2)]

In [78]:
df_filtered = df_overlap_filterd[
    (df_overlap_filterd['marker1'].isin(df_mp['marker_symbol'])) &
    (df_overlap_filterd['marker2'].isin(df_mp['marker_symbol']))
]

In [79]:
df_merged = pd.merge(df_filtered, df_mp, how='inner', left_on='marker1', right_on='marker_symbol')

print(len(df_merged))

df_merged.head(3)

32


,marker1,marker2,overlap_ratio,overlapped_mp_number,overlapped_mp,marker1_mp,marker2_mp,marker_symbol,effect_size
0,Dnmt3l,Cntd1,0.600,3,female infertility\nmale infertility\nsmall te...,female infertility\nincreased bone mineral den...,female infertility\nmale infertility\nsmall te...,Dnmt3l,1
1,Nphp1,Pgap6,0.600,3,abnormal skin morphology\nmale infertility\nsm...,abnormal skin morphology\nabnormal uterus morp...,abnormal skin morphology\nmale infertility\nsm...,Nphp1,1
2,Spidr,Kash5,0.545,6,decreased bone mineral content\ndecreased bone...,abnormal eye posterior chamber depth\ndecrease...,abnormal bone structure\ndecreased bone minera...,Spidr,1


### 出力

In [71]:
df_network = df_merged[["marker1", "marker2", "effect_size", "overlap_ratio", "marker1_mp", "marker2_mp", "overlapped_mp"]]

df_network.columns = ["from", "to", "node_size", "edge_size", "from_mp", "to_mp", "edge_mp"]
print(len(df_network))
print(df_network.head(3))


32
     from     to  node_size  edge_size  \
0  Dnmt3l  Cntd1          1      0.600   
1   Nphp1  Pgap6          1      0.600   
2   Spidr  Kash5          1      0.545   

                                             from_mp  \
0  female infertility\nincreased bone mineral den...   
1  abnormal skin morphology\nabnormal uterus morp...   
2  abnormal eye posterior chamber depth\ndecrease...   

                                               to_mp  \
0  female infertility\nmale infertility\nsmall te...   
1  abnormal skin morphology\nmale infertility\nsm...   
2  abnormal bone structure\ndecreased bone minera...   

                                             edge_mp  
0  female infertility\nmale infertility\nsmall te...  
1  abnormal skin morphology\nmale infertility\nsm...  
2  decreased bone mineral content\ndecreased bone...  


In [72]:
df_network.to_csv(Path("data", "network.csv"), index=False)

In [41]:
# %%bash

# python scripts/vis_network.py